<a href="https://colab.research.google.com/github/bolinocroustibat/movies-palettes/blob/main/movies_palettes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import numpy as np
import cv2
import glob
import hashlib
import json
import os
import re
import sqlite3
import time
from datetime import datetime, timezone
from dateutil import parser
from multiprocessing import Pool, cpu_count
from pathlib import Path, PosixPath
from PIL import Image
from skimage import feature, color
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm

# Configuration constants

*   **`RESIZE_W` and `RESIZE_H` (Reduced Frame Size)**

  This determines the resolution of each frame after downsampling. A reasonable value should:
    - Retain enough details for meaningful clustering.
    - Eliminate excessive computational overhead from large frame sizes.

  Suggested Value: 160 x 90 (W x H)
  - This keeps the aspect ratio of typical widescreen content (16:9).
  -	At 160x90, you get 14,400 pixels per frame, which is sufficient for clustering dominant colors while ensuring faster processing.

*  **`FRAME_SKIP` (Frames Skipped)**

    This determines how many frames you skip between processed frames.
  -	Movies at 24fps typically don’t have significant color changes frame-by-frame.
  -	A higher skip value reduces computation but might miss some rapid scene changes.

  Suggested Value: `FRAME_SKIP = 60`
 	- This processes 1 frame per 2.5 seconds (approx.), enough to capture major color changes without excessive redundancy.
 	-	For a 90-minute movie:
 	   - Total frames at 24fps:  90 x 60 x 24 = 129,600
 	   - With `FRAME_SKIP = 60`, you’ll process around  129,600 / 60 = 2,160  frames per movie.

*  **`BATCH_SIZE` (Frames Processed at Once)**

  This determines the number of frames processed in a single batch.
 	-	Larger batch sizes are computationally efficient as you can leverage batch processing in libraries like OpenCV.
 	-	However, too large a batch size might lead to memory limitations.

  Suggested Values:
  - For CPU processing, you can try `BATCH_SIZE = 20`.
  - For an NVIDIA A100 GPU, which has 40/80GB of VRAM depending on the variant, you can use a significantly larger batch, like `BATCH_SIZE = 32`



In [ ]:
CLUSTERS_NB = 10
CLUSTERS_NB_BW = 4

FRAME_SKIP = 60  # Process one frame out of every FRAME_SKIP frames in order to speed up the process
# Resize the image to RESIZE_W x RESIZE_H pixels in order to reduce complexity
RESIZE_W = 160
RESIZE_H = 90
BATCH_SIZE = 32  # Number of frames to process in each batch

METHOD = "cv2" # "cv2" or "sklearn". sklearn is supposedly more precise, but cv2 seems to give better results
COLOR_SPACE = "RGB"  # "RGB" or "LAB"
SATURATION_FACTOR = 1.0  # Default value of 1.0 means no saturation adjustment

# Recalculate a new palette even if it already has one if the palette is older than this.
# Put None if you want to recalculate no matter what
RECALC_IF_BEFORE = None
# Calculate only a specific movie
MOVIE_TO_PROCESS = "Dick Tracy"  # Set to None to process all movies

# Mount database from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

SQLITE_FILE_PATH = Path("/content/drive/MyDrive/MOVIES/movies.db")

# Function to get runtime type

In [ ]:
def get_runtype_type() -> str:
  if int(os.environ.get("COLAB_GPU", 0)) > 0:
    return "GPU"
  elif "TPU_DRIVER_MODE" in os.environ and os.environ["TPU_DRIVER_MODE"] == "tpu":
    return "TPU"
  elif "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
    return "TPU"
  return "unknown"

# Function to build a UUID

In [ ]:
def generate_palette_id(title: str, calculation_date: str) -> str:
    """Generate a short unique identifier for a palette"""
    unique_string = f"{title}_{calculation_date}"
    hash_object = hashlib.md5(unique_string.encode())
    return hash_object.hexdigest()[:6]

# Main logic

This is the main logic here. It might takes a few minutes to a few hours per movie file, depending on where it's run.

I suggest a GPU on Google Colab as the runtime.


In [ ]:
def is_black_and_white(cap: cv2.VideoCapture, threshold: float = 0.1) -> bool:
    """
    Determine if a movie is black and white based on the average saturation.

    Parameters:
    cap (cv2.VideoCapture): The video capture object.
    threshold (float): The saturation threshold below which the movie is considered B&W.

    Returns:
    bool: True if the movie is black and white, False otherwise.
    """
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_frames = min(total_frames, 100)  # Sample up to 100 frames

    saturation_values = []
    for i in range(0, total_frames, max(1, total_frames // sample_frames)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            continue
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        saturation = hsv[:, :, 1].mean() / 255.0  # Normalize to [0,1]
        saturation_values.append(saturation)

    average_saturation = np.mean(saturation_values)
    return average_saturation < threshold

In [ ]:
def get_dominant_colors_cv2(
    data: np.ndarray, clusters_nb: int, color_space: str = "RGB"
) -> tuple[np.ndarray, np.ndarray]:
    """OpenCV version with optional LAB color space, returns float64 centers"""
    if color_space == "LAB":
        lab_data = cv2.cvtColor(data.reshape(-1, 1, 3), cv2.COLOR_RGB2LAB).reshape(
            -1, 3
        )
        clustering_data = lab_data.astype(np.float32)
    else:
        clustering_data = data.astype(np.float32)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, centers = cv2.kmeans(
        clustering_data, clusters_nb, None, criteria, 10, flags
    )

    if color_space == "LAB":
        # Convert to uint8 for cv2.cvtColor, then back to float64
        centers_rgb = cv2.cvtColor(
            centers.reshape(-1, 1, 3).astype(np.uint8), cv2.COLOR_LAB2RGB
        )
        centers = centers_rgb.reshape(-1, 3).astype(np.float64)
    else:
        # Just convert to float64
        centers = centers.astype(np.float64)

    return centers, labels.ravel()

def get_dominant_colors_sklearn(
    data: np.ndarray, clusters_nb: int, color_space: str = "RGB"
) -> tuple[np.ndarray, np.ndarray]:
    """
    Cluster pixels using k-means and return the dominant colors.
    Will automatically reduce number of clusters if not enough distinct clusters are found.

    Parameters:
    data (np.ndarray): A 2D array where each row is a pixel in RGB format.
    clusters_nb (int): The maximum number of clusters to form.

    Returns:
    tuple[np.ndarray, np.ndarray]: A tuple containing:
        - centers (np.ndarray): The RGB values of the cluster centers.
        - labels (np.ndarray): The label of the cluster each pixel belongs to.
    """
    if color_space == "LAB":
        # Convert to LAB color space
        lab_data = cv2.cvtColor(data.reshape(-1, 1, 3), cv2.COLOR_RGB2LAB).reshape(
            -1, 3
        )
        clustering_data = lab_data
    else:
        clustering_data = data

    kmeans = KMeans(n_clusters=clusters_nb, n_init="auto")
    labels = kmeans.fit_predict(clustering_data)
    centers = kmeans.cluster_centers_

    if color_space == "LAB":
        # Convert centers back to RGB
        centers_rgb = cv2.cvtColor(
            centers.reshape(-1, 1, 3).astype(np.uint8), cv2.COLOR_LAB2RGB
        )
        centers = centers_rgb.reshape(-1, 3)

    return centers, labels


def get_dominant_luminances(
    data: np.ndarray, clusters_nb: int
) -> tuple[np.ndarray, np.ndarray]:
    """
    Find the dominant luminance values in an image using k-means clustering.
    Returns these luminance values as grayscale RGB values (where R=G=B).

    Parameters:
    data (np.ndarray): Input image (can be color or grayscale)
    clusters_nb (int): Number of luminance levels to identify

    Returns:
    tuple[np.ndarray, np.ndarray]: A tuple containing:
        - centers (np.ndarray): Array of shape (clusters_nb, 3) where each row is
          a grayscale RGB value [v,v,v] representing a dominant luminance level
        - labels (np.ndarray): Array indicating which cluster each pixel belongs to
    """
    # Convert to grayscale if input is color image
    if len(data.shape) > 2:
        data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)

    # Reshape to 1D array for clustering
    data_reshaped = data.reshape(-1, 1).astype(np.float32)

    # Define k-means parameters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS

    # Perform k-means clustering on luminance values
    _, labels, centers = cv2.kmeans(
        data_reshaped, clusters_nb, None, criteria, 10, flags
    )

    # Sort centers from darkest to lightest
    sort_idx = np.argsort(centers.flatten())
    centers = centers[sort_idx]

    # Convert luminance values to RGB format (all channels equal)
    # e.g., luminance value 127 becomes [127, 127, 127]
    centers_rgb = np.column_stack([centers.flatten()] * 3)

    return centers_rgb, labels

In [ ]:
def process_batch(frames: list[np.ndarray], is_bw: bool) -> np.ndarray:
    """
    Process a batch of frames at once to extract dominant colors.

    Parameters:
    frames (list): List of frames to process
    is_bw (bool): Whether to process as black and white

    Returns:
    np.ndarray: Array of dominant colors found across all frames
    """
    # Stack all frames into a single array
    batch = np.stack(frames)

    # Resize all frames at once
    resized = np.array([cv2.resize(f, (RESIZE_W, RESIZE_H)) for f in batch])

    if is_bw:
        # Convert to grayscale and process all frames
        gray_batch = np.array([cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in resized])
        all_pixels = gray_batch.reshape(-1, 1)
        centers, labels = get_dominant_luminances(all_pixels, CLUSTERS_NB_BW)
    else:
        # Convert to RGB and process all frames
        rgb_batch = np.array([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in resized])

        # Apply saturation adjustment only if factor is not 1.0
        if SATURATION_FACTOR != 1.0:
            hsv_batch = np.array([cv2.cvtColor(f, cv2.COLOR_RGB2HSV) for f in rgb_batch])
            hsv_batch[:, :, :, 1] = hsv_batch[:, :, :, 1] * SATURATION_FACTOR
            hsv_batch[:, :, :, 1] = np.clip(hsv_batch[:, :, :, 1], 0, 255)
            rgb_batch = np.array([cv2.cvtColor(f, cv2.COLOR_HSV2RGB) for f in hsv_batch])

        # Reshape to process all pixels from all frames
        all_pixels = rgb_batch.reshape(-1, 3)

        if METHOD == "cv2":
            centers, labels = get_dominant_colors_cv2(all_pixels, CLUSTERS_NB, COLOR_SPACE)
        elif METHOD == "sklearn":
            centers, labels = get_dominant_colors_sklearn(all_pixels, CLUSTERS_NB, COLOR_SPACE)
        else:
            raise ValueError(f"Unknown method: {METHOD}")

    # Sort clusters by frequency
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_idx = np.argsort(-counts)  # Sort in descending order
    centers = centers[sorted_idx]

    return centers


In [ ]:
def process_movie(movie_title: str, file_path: Path) -> dict | None:
    """
    Calculate the palette for a movie.

    This function processes a movie file to extract its dominant color palette.
    For black and white movies, it extracts dominant luminance values instead.
    The processing is done in batches to improve performance.

    Parameters:
    - path: Path to the movie file
    - title: Movie title
    Returns:
    dict: The new palette entry
    """
    start_time: float = time.time()

    pbar.set_description(f'Processing movie "{movie_title}"...')

    cap = cv2.VideoCapture(str(file_path))
    frames_count: int = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_process: int = frames_count // FRAME_SKIP
    if frames_to_process < 1:
        print(f'Movie "{movie_title}" has less than {FRAME_SKIP} frames, skipping...')
        return None

    # Detect if movie is black and white
    is_bw: bool = is_black_and_white(cap)
    if is_bw:
        print(f'Movie "{movie_title}" is black and white, with {frames_count} frames.')
    else:
        print(f'Movie "{movie_title}" is color, with {frames_count} frames.')

    frame_nb = 0
    all_centers = []
    frames_processed = 0

    with tqdm(total=frames_to_process, desc="Frames processed") as frame_pbar:
        while frame_nb < frames_count:
            batch_frames = []
            frames_read = 0

            # Calculate how many frames we still need to process
            remaining_frames = frames_to_process - frames_processed
            current_batch_size = min(BATCH_SIZE, remaining_frames)

            # Read up to current_batch_size frames
            for _ in range(current_batch_size):
                if frame_nb >= frames_count:
                    break

                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nb)
                status, frame = cap.read()

                if not status:
                    frame_nb += FRAME_SKIP
                    continue

                batch_frames.append(frame)
                frames_read += 1
                frame_nb += FRAME_SKIP

            if not batch_frames:
                break

            # Process batch and extend results
            batch_centers = process_batch(batch_frames, is_bw)
            all_centers.extend(batch_centers)
            frame_pbar.update(frames_read)
            frames_processed += frames_read

    if len(all_centers) == 0:
        print(f"No colors extracted for {movie_title}. Skipping.")
        return None

    # Combine all centers and do final clustering
    all_centers = np.vstack(all_centers)
    kmeans = KMeans(
        n_clusters=CLUSTERS_NB if not is_bw else CLUSTERS_NB_BW, n_init="auto"
    )
    kmeans.fit(all_centers)
    final_colors: np.ndarray = kmeans.cluster_centers_
    final_palette = [tuple(map(int, c)) for c in final_colors]

    # Calculate the total processing duration
    end_time: float = time.time()
    duration: int = int(end_time - start_time)
    # Calculation date
    now: str = datetime.now(timezone.utc).isoformat()

    return {
        "id": generate_palette_id(movie_title, now),
        "calculation_date": now,
        "calculation_duration_seconds": duration,
        "is_black_and_white": is_bw,
        "colors": final_palette,
        "clusters_nb": CLUSTERS_NB_BW if is_bw else CLUSTERS_NB,
        "frame_skip": FRAME_SKIP,
        "resize_width": RESIZE_W,
        "resize_height": RESIZE_H,
        "batch_size": BATCH_SIZE,
        "clustering_method": METHOD,
        "color_space": COLOR_SPACE,
        "saturation_factor": SATURATION_FACTOR,
        "runtime": get_runtype_type()
    }


# Display palette

In [ ]:
def display_palette(palette: dict) -> None:
    # Display metadata
    for k, v in palette.items():
        if k != "colors":  # Skip colors as we'll display them separately
            print(f'{k}: {v}')

    # Display color swatches
    for c in palette["colors"]:
        img = Image.new(mode="RGB", size=(200, 30), color=tuple(c))
        display(img)

# Final loop through movies

In [ ]:
with sqlite3.connect(SQLITE_FILE_PATH) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT
            m.id,
            m.title,
            m.path,
            JSON_GROUP_ARRAY(p.calculation_date) as palettes
        FROM movies m
        LEFT JOIN palettes p ON m.id = p.movie_id
        WHERE m.type = 'movie'
        GROUP BY m.id, m.title;
    """)
    movies = cursor.fetchall()
    print(f'Found {len(movies)} movies to process.')

    for m in (pbar := tqdm(movies)):
        # Skip movies not matching MOVIE_TO_PROCESS, if specified
        if MOVIE_TO_PROCESS and m[1] != MOVIE_TO_PROCESS:
            continue

        # Check if movie already has palettes and if any are newer than RECALC_IF_BEFORE
        if RECALC_IF_BEFORE is not None and m[3] is not None and m[3] != '[null]':
            calculation_dates: list[str] = json.loads(m[3])
            skip_movie: bool = False
            for date_str in calculation_dates:
                if date_str is not None and date_str != "":
                    calculation_date = parser.parse(date_str)
                    if calculation_date > RECALC_IF_BEFORE:
                        skip_movie = True
                        break  # No need to check further dates

            if skip_movie:
                # tqdm.write(f'"{m[1]}" already has a recent color palette, skipping...')
                continue

        if not m[2]:
            # tqdm.write(f'"{m[1]}" has no filepath, skipping...')
            continue

        else:
            palette: dict = process_movie(movie_title=m[1], file_path=Path(m[2]))
            if palette is None:
                continue

            # Debug: display new palette
            tqdm.write(f'Calculated palette for "{m[1]}"')
            display_palette(palette)

            # Insert the new palette into the DB
            sql_query = """
            INSERT OR REPLACE INTO palettes (
                id,
                movie_id,
                calculation_date,
                calculation_duration_seconds,
                is_black_and_white,
                colors,
                clusters_nb,
                frame_skip,
                resize_width,
                resize_height,
                batch_size,
                clustering_method,
                color_space,
                saturation_factor,
                runtime
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            data_to_insert = (
                palette["id"],
                m[0],
                palette["calculation_date"],
                palette["calculation_duration_seconds"],
                palette["is_black_and_white"],
                json.dumps(palette.get("colors", [])),
                palette["clusters_nb"],
                palette["frame_skip"],
                palette["resize_width"],
                palette["resize_height"],
                palette["batch_size"],
                palette["clustering_method"],
                palette["color_space"],
                palette["saturation_factor"],
                palette["runtime"],
            )
            cursor.execute(sql_query, data_to_insert)
            conn.commit()
